# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [5]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

#sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')

# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [18]:
# Filtrando as colunas relevantes para o cálculo da média da idade da mãe/pai
colunas_relevantes = ['munResNome', 'IDADEMAE', 'IDADEPAI']

# Filtrando os dados
dados_filtrados = sinasc_raw[colunas_relevantes]

# Fazendo uma cópia do DataFrame filtrado
dados_filtrados_copia = dados_filtrados.copy()

# Renomeando a coluna 'munResNome'
dados_filtrados_copia.rename(columns={'munResNome': 'Município(munResNome)'}, inplace=True)

# Calculando a média da idade das mães/pais por município usando operações de agregação e grupos
media_idade_mae_pai = dados_filtrados_copia.groupby('Município(munResNome)').agg({'IDADEMAE': np.mean, 'IDADEPAI': np.mean})

# Arredondando as idades para números inteiros
media_idade_mae_pai = media_idade_mae_pai.round(0).astype(int)

# PRINT
print(media_idade_mae_pai)

# Visualizar apenas as médias de idade das mães
#media_idade_mae = dados_filtrados.groupby('munResNome')['IDADEMAE'].mean()
#print(media_idade_mae)

# Visualizar apenas as médias de idade dos pais
#media_idade_pai = dados_filtrados.groupby('munResNome')['IDADEPAI'].mean()
#print(media_idade_pai)


                           IDADEMAE  IDADEPAI
Município(munResNome)                        
Alta Floresta D'Oeste            26        30
Alto Alegre dos Parecis          25        29
Alto Paraíso                     25        29
Alvorada D'Oeste                 26        31
Ariquemes                        26        32
Buritis                          26        31
Cabixi                           26        34
Cacaulândia                      26        36
Cacoal                           27        31
Campo Novo de Rondônia           25        30
Candeias do Jamari               25        30
Castanheiras                     27        30
Cerejeiras                       27        32
Chupinguaia                      25        30
Colorado do Oeste                28        34
Corumbiara                       25        32
Costa Marques                    24        31
Cujubim                          24        31
Espigão D'Oeste                  26        31
Governador Jorge Teixeira        2

### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [49]:
# Definindo a data de aniversário 
data_aniversario = '2019-02-24'

# Filtrando os dados para a data de aniversário
dados_aniversario = sinasc_raw[sinasc_raw['DTNASC'] == data_aniversario].dropna(subset=['SEXO','ESCMAE','PESO'])

# Agrupando os dados por sexo e faixa de escolaridade da mãe, e calculando o peso médio
peso_medio_por_sexo_escolaridade = dados_aniversario.groupby(['SEXO', 'ESCMAE'])['PESO'].mean()

# PRINT
print(peso_medio_por_sexo_escolaridade)


SEXO       ESCMAE         
Feminino   1 a 3 anos         2995.000000
           12 anos ou mais    3342.142857
           4 a 7 anos         3094.285714
           8 a 11 anos        3004.375000
Masculino  4 a 7 anos         3105.000000
           8 a 11 anos        3313.500000
Name: PESO, dtype: float64


### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [60]:
# Encontrando município com menor número de bebes nascidos
municipio_menos_bebes = sinasc_raw['munResNome'].value_counts().idxmin()

# Excluindo o município com o menor número de bebês, pois ele foi considerado um município ignorado nos dados.
sinasc_raw = sinasc_raw[sinasc_raw['munResNome'] != municipio_menos_bebes]
municipio_segundo_menos_bebes = sinasc_raw['munResNome'].value_counts().idxmin()

# Filtrando os dados para o segundo município com o menor número de bebês
dados_municipio_segundo_menos_bebes = sinasc_raw[sinasc_raw['munResNome'] == municipio_segundo_menos_bebes]

# Calculando a idade média, máxima e mínima das mães no segundo município
idade_media_maes_segundo = dados_municipio_segundo_menos_bebes['IDADEMAE'].mean()
idade_media_maes_segundo = int(idade_media_maes_segundo)  
idade_maxima_maes_segundo = int(dados_municipio_segundo_menos_bebes['IDADEMAE'].max())  
idade_minima_maes_segundo = int(dados_municipio_segundo_menos_bebes['IDADEMAE'].min())  

# Calculando a idade média, máxima e mínima dos pais no segundo município
idade_media_pais_segundo = dados_municipio_segundo_menos_bebes['IDADEPAI'].mean()
idade_media_pais_segundo = int(idade_media_pais_segundo) 
idade_maxima_pais_segundo = int(dados_municipio_segundo_menos_bebes['IDADEPAI'].max())  
idade_minima_pais_segundo = int(dados_municipio_segundo_menos_bebes['IDADEPAI'].min())  

# PRINT
print("Município com menos bebês nascidos em 2019:", municipio_segundo_menos_bebes)
print()
print("Idade média das mães no segundo município com menos bebês:", idade_media_maes_segundo)
print("Idade máxima das mães no segundo município com menos bebês:", idade_maxima_maes_segundo)
print("Idade mínima das mães no segundo município com menos bebês:", idade_minima_maes_segundo)
print()
print("Idade média dos pais no segundo município com menos bebês:", idade_media_pais_segundo)
print("Idade máxima dos pais no segundo município com menos bebês:", idade_maxima_pais_segundo)
print("Idade mínima dos pais no segundo município com menos bebês:", idade_minima_pais_segundo)


Município com menos bebês nascidos em 2019: Pimenteiras do Oeste

Idade média das mães no segundo município com menos bebês: 25
Idade máxima das mães no segundo município com menos bebês: 40
Idade mínima das mães no segundo município com menos bebês: 14

Idade média dos pais no segundo município com menos bebês: 35
Idade máxima dos pais no segundo município com menos bebês: 45
Idade mínima dos pais no segundo município com menos bebês: 25


### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [64]:
# Filtrando os dados para o mês de MARÇO
dados_marco = sinasc_raw[sinasc_raw['DTNASC'].str.startswith('2019-03')]

# Encontrando o município que teve o maior número de bebês nascidos em MARÇO
municipio_mais_bebes_marco = dados_marco['munResNome'].value_counts().idxmax()

# Filtrando os dados para o município com o maior número de bebês nascidos em MARÇO
dados_municipio_mais_bebes_marco = dados_marco[dados_marco['munResNome'] == municipio_mais_bebes_marco]

# Calculando a quantidade de filhos vivos média, máxima e mínima no município de MARÇO
qtd_filhos_vivos_media_marco = int(dados_municipio_mais_bebes_marco['QTDFILVIVO'].mean())
qtd_filhos_vivos_maxima_marco = int(dados_municipio_mais_bebes_marco['QTDFILVIVO'].max())
qtd_filhos_vivos_minima_marco = int(dados_municipio_mais_bebes_marco['QTDFILVIVO'].min())

# Calculando a idade média, máxima e mínima dos pais no município de MARÇO
idade_media_pais_marco = int(dados_municipio_mais_bebes_marco['IDADEPAI'].mean())
idade_maxima_pais_marco = int(dados_municipio_mais_bebes_marco['IDADEPAI'].max())
idade_minima_pais_marco = int(dados_municipio_mais_bebes_marco['IDADEPAI'].min())

# PRINT
print("Município com mais bebês nascidos em março:", municipio_mais_bebes_marco)
print()
print("Quantidade de filhos vivos média no município em março:", qtd_filhos_vivos_media_marco)
print("Quantidade de filhos vivos máxima no município em março:", qtd_filhos_vivos_maxima_marco)
print("Quantidade de filhos vivos mínima no município em março:", qtd_filhos_vivos_minima_marco)
print()
print("Idade média dos pais no município em março:", idade_media_pais_marco)
print("Idade máxima dos pais no município em março:", idade_maxima_pais_marco)
print("Idade mínima dos pais no município em março:", idade_minima_pais_marco)


Município com mais bebês nascidos em março: Porto Velho

Quantidade de filhos vivos média no município em março: 1
Quantidade de filhos vivos máxima no município em março: 9
Quantidade de filhos vivos mínima no município em março: 0

Idade média dos pais no município em março: 34
Idade máxima dos pais no município em março: 62
Idade mínima dos pais no município em março: 19


### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

**RESPOSTA:**
* É possível observar que a média de idade das mães varia entre 24 e 28 anos, enquanto a média de idade dos pais varia entre 29 e 42 anos. Podemos inferir que há uma tendência geral de maternidade em idades mais jovens para as mães e em idades mais avançadas para os pais.

* O município com menos bebês nascidos em 2019 foi "Pimenteiras do Oeste". Além disso, observamos que a idade média das mães nesse município é relativamente jovem, com uma média de 25 anos, e a idade média dos pais é de 35 anos, podendo indicar uma população mais jovem nesse município.



In [6]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [7]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [4]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')